In [1]:
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import conjuntoDeDatos
import os

#Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
batch_size = 32

#Prepare input data
classes = ['dog','cat']
num_classes = len(classes)

# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 128
num_channels = 3
train_path=os.path.join(os.path.realpath('.'),'imagenes/train/')
# We shall load all the training and validation images and labels into memory using openCV and use that during training
# data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)
data = conjuntoDeDatos.leerDatosDeEntrenamiento(train_path, img_size, classes,validation_size)

print("Complete reading input data. Will Now print a snippet of it")
print("Numero de imagenes en el conjunto de entrenamiento:\t{}".format(len(data.entrenamiento.etiquetas)))
print("Numero de imagenes en el conjunto de validacion :\t{}".format(len(data.validacion.etiquetas)))


clase dog, indice 0, path /notebooks/CNNcatdogs/imagenes/train/dog.*.jpg, files number 6891, max images 10
progress, read 999 of 6891
progress, read 1999 of 6891
progress, read 2999 of 6891
progress, read 3999 of 6891
progress, read 4999 of 6891
progress, read 5999 of 6891
clase cat, indice 1, path /notebooks/CNNcatdogs/imagenes/train/cat.*.jpg, files number 12500, max images 10
progress, read 999 of 12500
progress, read 1999 of 12500
progress, read 2999 of 12500
progress, read 3999 of 12500
progress, read 4999 of 12500
progress, read 5999 of 12500
progress, read 6999 of 12500
progress, read 7999 of 12500
progress, read 8999 of 12500
progress, read 9999 of 12500
progress, read 10999 of 12500
progress, read 11999 of 12500
End to get images
Complete reading input data. Will Now print a snippet of it
Numero de imagenes en el conjunto de entrenamiento:	15481
Numero de imagenes en el conjunto de validacion :	3870


In [2]:

session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

##Network graph params
filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128

def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))



def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer

    

def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer


def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer


layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 


def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

total_iterations = 0

saver = tf.train.Saver()
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):
        print("-- iteracion # "+str(i+1)+" con imagenes # "+str(batch_size))
        x_batch, y_true_batch, _, cls_batch = data.entrenamiento.siguienteLote(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.validacion.siguienteLote(batch_size)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.entrenamiento.recuento/batch_size) == 0: 
            print("mostrar epoca "+str(int(data.entrenamiento.recuento/batch_size)))
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.entrenamiento.recuento/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, './model/codigoOriginal/dogs-cats-model_16_04_2018_v.1') 


    total_iterations += num_iteration

train(num_iteration=3000)

siguiente lote de entrenamiento, inicio: 0, fin: 32
siguiente lote de validacion, inicio: 0, fin: 32
Training Epoch 1 --- Training Accuracy:  62.5%, Validation Accuracy:  59.4%,  Validation Loss: 0.688
siguiente lote de entrenamiento, inicio: 32, fin: 64
siguiente lote de validacion, inicio: 32, fin: 64
siguiente lote de entrenamiento, inicio: 64, fin: 96
siguiente lote de validacion, inicio: 64, fin: 96
siguiente lote de entrenamiento, inicio: 96, fin: 128
siguiente lote de validacion, inicio: 96, fin: 128
siguiente lote de entrenamiento, inicio: 128, fin: 160
siguiente lote de validacion, inicio: 128, fin: 160
siguiente lote de entrenamiento, inicio: 160, fin: 192
siguiente lote de validacion, inicio: 160, fin: 192
siguiente lote de entrenamiento, inicio: 192, fin: 224
siguiente lote de validacion, inicio: 192, fin: 224
siguiente lote de entrenamiento, inicio: 224, fin: 256
siguiente lote de validacion, inicio: 224, fin: 256
siguiente lote de entrenamiento, inicio: 256, fin: 288
sigu

siguiente lote de entrenamiento, inicio: 2400, fin: 2432
siguiente lote de validacion, inicio: 2400, fin: 2432
siguiente lote de entrenamiento, inicio: 2432, fin: 2464
siguiente lote de validacion, inicio: 2432, fin: 2464
siguiente lote de entrenamiento, inicio: 2464, fin: 2496
siguiente lote de validacion, inicio: 2464, fin: 2496
siguiente lote de entrenamiento, inicio: 2496, fin: 2528
siguiente lote de validacion, inicio: 2496, fin: 2528
siguiente lote de entrenamiento, inicio: 2528, fin: 2560
siguiente lote de validacion, inicio: 2528, fin: 2560
siguiente lote de entrenamiento, inicio: 2560, fin: 2592
siguiente lote de validacion, inicio: 2560, fin: 2592
siguiente lote de entrenamiento, inicio: 2592, fin: 2624
siguiente lote de validacion, inicio: 2592, fin: 2624
siguiente lote de entrenamiento, inicio: 2624, fin: 2656
siguiente lote de validacion, inicio: 2624, fin: 2656
siguiente lote de entrenamiento, inicio: 2656, fin: 2688
siguiente lote de validacion, inicio: 2656, fin: 2688
s

siguiente lote de entrenamiento, inicio: 4800, fin: 4832
siguiente lote de validacion, inicio: 960, fin: 992
siguiente lote de entrenamiento, inicio: 4832, fin: 4864
siguiente lote de validacion, inicio: 992, fin: 1024
siguiente lote de entrenamiento, inicio: 4864, fin: 4896
siguiente lote de validacion, inicio: 1024, fin: 1056
siguiente lote de entrenamiento, inicio: 4896, fin: 4928
siguiente lote de validacion, inicio: 1056, fin: 1088
siguiente lote de entrenamiento, inicio: 4928, fin: 4960
siguiente lote de validacion, inicio: 1088, fin: 1120
siguiente lote de entrenamiento, inicio: 4960, fin: 4992
siguiente lote de validacion, inicio: 1120, fin: 1152
siguiente lote de entrenamiento, inicio: 4992, fin: 5024
siguiente lote de validacion, inicio: 1152, fin: 1184
siguiente lote de entrenamiento, inicio: 5024, fin: 5056
siguiente lote de validacion, inicio: 1184, fin: 1216
siguiente lote de entrenamiento, inicio: 5056, fin: 5088
siguiente lote de validacion, inicio: 1216, fin: 1248
sigu

siguiente lote de entrenamiento, inicio: 7168, fin: 7200
siguiente lote de validacion, inicio: 3328, fin: 3360
siguiente lote de entrenamiento, inicio: 7200, fin: 7232
siguiente lote de validacion, inicio: 3360, fin: 3392
siguiente lote de entrenamiento, inicio: 7232, fin: 7264
siguiente lote de validacion, inicio: 3392, fin: 3424
siguiente lote de entrenamiento, inicio: 7264, fin: 7296
siguiente lote de validacion, inicio: 3424, fin: 3456
siguiente lote de entrenamiento, inicio: 7296, fin: 7328
siguiente lote de validacion, inicio: 3456, fin: 3488
siguiente lote de entrenamiento, inicio: 7328, fin: 7360
siguiente lote de validacion, inicio: 3488, fin: 3520
siguiente lote de entrenamiento, inicio: 7360, fin: 7392
siguiente lote de validacion, inicio: 3520, fin: 3552
siguiente lote de entrenamiento, inicio: 7392, fin: 7424
siguiente lote de validacion, inicio: 3552, fin: 3584
siguiente lote de entrenamiento, inicio: 7424, fin: 7456
siguiente lote de validacion, inicio: 3584, fin: 3616
s

siguiente lote de entrenamiento, inicio: 9568, fin: 9600
siguiente lote de validacion, inicio: 1888, fin: 1920
siguiente lote de entrenamiento, inicio: 9600, fin: 9632
siguiente lote de validacion, inicio: 1920, fin: 1952
siguiente lote de entrenamiento, inicio: 9632, fin: 9664
siguiente lote de validacion, inicio: 1952, fin: 1984
siguiente lote de entrenamiento, inicio: 9664, fin: 9696
siguiente lote de validacion, inicio: 1984, fin: 2016
siguiente lote de entrenamiento, inicio: 9696, fin: 9728
siguiente lote de validacion, inicio: 2016, fin: 2048
siguiente lote de entrenamiento, inicio: 9728, fin: 9760
siguiente lote de validacion, inicio: 2048, fin: 2080
siguiente lote de entrenamiento, inicio: 9760, fin: 9792
siguiente lote de validacion, inicio: 2080, fin: 2112
siguiente lote de entrenamiento, inicio: 9792, fin: 9824
siguiente lote de validacion, inicio: 2112, fin: 2144
siguiente lote de entrenamiento, inicio: 9824, fin: 9856
siguiente lote de validacion, inicio: 2144, fin: 2176
s

...

siguiente lote de entrenamiento, inicio: 8224, fin: 8256
siguiente lote de validacion, inicio: 1024, fin: 1056
siguiente lote de entrenamiento, inicio: 8256, fin: 8288
siguiente lote de validacion, inicio: 1056, fin: 1088
siguiente lote de entrenamiento, inicio: 8288, fin: 8320
siguiente lote de validacion, inicio: 1088, fin: 1120
siguiente lote de entrenamiento, inicio: 8320, fin: 8352
siguiente lote de validacion, inicio: 1120, fin: 1152
siguiente lote de entrenamiento, inicio: 8352, fin: 8384
siguiente lote de validacion, inicio: 1152, fin: 1184
siguiente lote de entrenamiento, inicio: 8384, fin: 8416
siguiente lote de validacion, inicio: 1184, fin: 1216
siguiente lote de entrenamiento, inicio: 8416, fin: 8448
siguiente lote de validacion, inicio: 1216, fin: 1248
siguiente lote de entrenamiento, inicio: 8448, fin: 8480
siguiente lote de validacion, inicio: 1248, fin: 1280
siguiente lote de entrenamiento, inicio: 8480, fin: 8512
siguiente lote de validacion, inicio: 1280, fin: 1312
s

siguiente lote de entrenamiento, inicio: 10592, fin: 10624
siguiente lote de validacion, inicio: 3392, fin: 3424
siguiente lote de entrenamiento, inicio: 10624, fin: 10656
siguiente lote de validacion, inicio: 3424, fin: 3456
siguiente lote de entrenamiento, inicio: 10656, fin: 10688
siguiente lote de validacion, inicio: 3456, fin: 3488
siguiente lote de entrenamiento, inicio: 10688, fin: 10720
siguiente lote de validacion, inicio: 3488, fin: 3520
siguiente lote de entrenamiento, inicio: 10720, fin: 10752
siguiente lote de validacion, inicio: 3520, fin: 3552
siguiente lote de entrenamiento, inicio: 10752, fin: 10784
siguiente lote de validacion, inicio: 3552, fin: 3584
siguiente lote de entrenamiento, inicio: 10784, fin: 10816
siguiente lote de validacion, inicio: 3584, fin: 3616
siguiente lote de entrenamiento, inicio: 10816, fin: 10848
siguiente lote de validacion, inicio: 3616, fin: 3648
siguiente lote de entrenamiento, inicio: 10848, fin: 10880
siguiente lote de validacion, inicio:

siguiente lote de entrenamiento, inicio: 12960, fin: 12992
siguiente lote de validacion, inicio: 1920, fin: 1952
siguiente lote de entrenamiento, inicio: 12992, fin: 13024
siguiente lote de validacion, inicio: 1952, fin: 1984
siguiente lote de entrenamiento, inicio: 13024, fin: 13056
siguiente lote de validacion, inicio: 1984, fin: 2016
siguiente lote de entrenamiento, inicio: 13056, fin: 13088
siguiente lote de validacion, inicio: 2016, fin: 2048
siguiente lote de entrenamiento, inicio: 13088, fin: 13120
siguiente lote de validacion, inicio: 2048, fin: 2080
siguiente lote de entrenamiento, inicio: 13120, fin: 13152
siguiente lote de validacion, inicio: 2080, fin: 2112
siguiente lote de entrenamiento, inicio: 13152, fin: 13184
siguiente lote de validacion, inicio: 2112, fin: 2144
siguiente lote de entrenamiento, inicio: 13184, fin: 13216
siguiente lote de validacion, inicio: 2144, fin: 2176
siguiente lote de entrenamiento, inicio: 13216, fin: 13248
siguiente lote de validacion, inicio:

siguiente lote de entrenamiento, inicio: 15296, fin: 15328
siguiente lote de validacion, inicio: 416, fin: 448
siguiente lote de entrenamiento, inicio: 15328, fin: 15360
siguiente lote de validacion, inicio: 448, fin: 480
siguiente lote de entrenamiento, inicio: 15360, fin: 15392
siguiente lote de validacion, inicio: 480, fin: 512
siguiente lote de entrenamiento, inicio: 15392, fin: 15424
siguiente lote de validacion, inicio: 512, fin: 544
siguiente lote de entrenamiento, inicio: 15424, fin: 15456
siguiente lote de validacion, inicio: 544, fin: 576
siguiente lote de entrenamiento, inicio: 0, fin: 32
siguiente lote de validacion, inicio: 576, fin: 608
Training Epoch 7 --- Training Accuracy:  71.9%, Validation Accuracy:  75.0%,  Validation Loss: 0.533
siguiente lote de entrenamiento, inicio: 32, fin: 64
siguiente lote de validacion, inicio: 608, fin: 640
siguiente lote de entrenamiento, inicio: 64, fin: 96
siguiente lote de validacion, inicio: 640, fin: 672
siguiente lote de entrenamient

siguiente lote de entrenamiento, inicio: 2208, fin: 2240
siguiente lote de validacion, inicio: 2784, fin: 2816
siguiente lote de entrenamiento, inicio: 2240, fin: 2272
siguiente lote de validacion, inicio: 2816, fin: 2848
siguiente lote de entrenamiento, inicio: 2272, fin: 2304
siguiente lote de validacion, inicio: 2848, fin: 2880
siguiente lote de entrenamiento, inicio: 2304, fin: 2336
siguiente lote de validacion, inicio: 2880, fin: 2912
siguiente lote de entrenamiento, inicio: 2336, fin: 2368
siguiente lote de validacion, inicio: 2912, fin: 2944
siguiente lote de entrenamiento, inicio: 2368, fin: 2400
siguiente lote de validacion, inicio: 2944, fin: 2976
siguiente lote de entrenamiento, inicio: 2400, fin: 2432
siguiente lote de validacion, inicio: 2976, fin: 3008
siguiente lote de entrenamiento, inicio: 2432, fin: 2464
siguiente lote de validacion, inicio: 3008, fin: 3040
siguiente lote de entrenamiento, inicio: 2464, fin: 2496
siguiente lote de validacion, inicio: 3040, fin: 3072
s